# Tutorial 3: Performance Evaluation Framework

This tutorial provides a deep dive into the **le_eval** module — the performance
benchmarking framework for systematically measuring and comparing detector
performance across image datasets.

## What You Will Learn

1. **StringTable** — structured result storage and CSV export
2. **Performance Primitives** — `PerformanceResult`, `PerformanceMeasureBase`,
   `CVPerformanceMeasure`
3. **Data Providers** — loading images from files for benchmarking
4. **Writing Custom Tasks** — subclassing `CVPerformanceTask` in Python
5. **CVPerformanceTest** — the orchestrator for running benchmarks
6. **Result Analysis** — accumulating, visualizing, and exporting results
7. **Full Benchmark** — complete end-to-end example

## Prerequisites

- Completed **Tutorial 1 & 2** (or equivalent knowledge of `le_imgproc`,
  `le_edge`, `le_lsd`)
- Built the LE Python bindings: `bazel build //libs/...`
- Python kernel set to the project `.venv`

## 1. Setup & Imports

In [ ]:
import sys, pathlib, time, tempfile, os

# --- Locate workspace root and add Bazel output dirs to sys.path ---
workspace = pathlib.Path.cwd()
while not (workspace / "MODULE.bazel").exists():
    if workspace == workspace.parent:
        raise RuntimeError("Cannot find LineExtraction workspace root (MODULE.bazel)")
    workspace = workspace.parent

for lib in ["imgproc", "edge", "geometry", "eval", "lsd"]:
    p = workspace / f"bazel-bin/libs/{lib}/python"
    if p.exists():
        sys.path.insert(0, str(p))
    else:
        print(f"Warning: Not found: {p}  — run: bazel build //libs/{lib}/...")

sys.path.insert(0, str(workspace / "python"))

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# NOTE: Do NOT import cv2 — the LE bindings ship their own statically linked
# OpenCV build. Using pip's cv2 would cause symbol conflicts.

import le_eval
import le_imgproc
import le_edge
import le_lsd
import le_geometry
from lsfm.data import TestImages

print(f"Workspace: {workspace}")
print(f"le_eval loaded: {len([x for x in dir(le_eval) if not x.startswith('_')])} symbols")

In [ ]:
# --- Common helpers ---

def show_images(images, titles, cmap="gray", figsize=None):
    """Show a list of images side by side."""
    n = len(images)
    if figsize is None:
        figsize = (4 * n, 4)
    fig, axes = plt.subplots(1, n, figsize=figsize)
    if n == 1:
        axes = [axes]
    for ax, img, title in zip(axes, images, titles):
        ax.imshow(img, cmap=cmap if img.ndim == 2 else None)
        ax.set_title(title, fontsize=10)
        ax.axis("off")
    plt.tight_layout()
    plt.show()

print("Helpers ready.")

---
## 2. StringTable — Structured Result Storage

`StringTable` is a simple 2D table of strings used for collecting and
displaying benchmark results. It supports indexing, row/column extraction,
transposition, and CSV export.

In [ ]:
# Construction: StringTable(rows, cols)
table = le_eval.StringTable(3, 4)
print(f"Empty table: {table.rows} rows × {table.cols} cols, size={table.size}")

# Set values via [row, col] indexing
headers = ["Detector", "Segments", "Avg Len", "Time (ms)"]
for c, h in enumerate(headers):
    table[0, c] = h

data = [
    ["LsdCC", "142", "23.5", "4.2"],
    ["LsdFGioi", "98", "31.2", "8.7"],
]
for r, row in enumerate(data):
    for c, val in enumerate(row):
        table[r + 1, c] = val

print(f"\nTable contents:")
print(repr(table))

In [ ]:
# Row and column extraction
print("Header row:", table.row(0))
print("Detector column:", table.col(0))

# To list (full 2D)
all_data = table.to_list()
print("\nFull data as list:")
for row in all_data:
    print(f"  {row}")

# Transpose
t = table.transpose()
print(f"\nTransposed: {t.rows}×{t.cols}")
print("First row of transposed:", t.row(0))

In [ ]:
# Save to CSV
csv_path = os.path.join(tempfile.gettempdir(), "le_results.csv")
table.save_csv(csv_path)

with open(csv_path) as f:
    print(f"CSV content ({csv_path}):")
    print(f.read())

os.unlink(csv_path)  # clean up

### Exercise 3.1

Create a `StringTable` with 5 rows and 3 columns. Fill the header row with
`["Name", "Value", "Unit"]`. Add 4 data rows describing different image
properties (e.g., width, height, channels, dtype). Print it, transpose it,
and save the transposed version to CSV.

In [ ]:
# TODO: Build a StringTable, fill it, transpose, save to CSV.

**Solution**

In [ ]:
t = le_eval.StringTable(5, 3)
t[0, 0] = "Name"; t[0, 1] = "Value"; t[0, 2] = "Unit"
t[1, 0] = "Width";    t[1, 1] = "1920";  t[1, 2] = "px"
t[2, 0] = "Height";   t[2, 1] = "1080";  t[2, 2] = "px"
t[3, 0] = "Channels"; t[3, 1] = "3";     t[3, 2] = "-"
t[4, 0] = "Dtype";    t[4, 1] = "uint8"; t[4, 2] = "-"

print(repr(t))
print()

tt = t.transpose()
print(f"Transposed ({tt.rows}×{tt.cols}):")
print(repr(tt))

csv_path = os.path.join(tempfile.gettempdir(), "image_props.csv")
tt.save_csv(csv_path)
with open(csv_path) as f:
    print(f.read())
os.unlink(csv_path)

---
## 3. Performance Primitives

The evaluation framework uses several types for collecting and computing
performance statistics.

### 3.1 PerformanceResult

In [ ]:
# PerformanceResult holds computed statistics: total, mean, stddev
result = le_eval.PerformanceResult()
print(f"Default: total={result.total}, mean={result.mean}, stddev={result.stddev}")

# Set values directly
result.total = 150.0
result.mean = 15.0
result.stddev = 2.3
print(f"Set:     total={result.total}, mean={result.mean}, stddev={result.stddev}")
print(repr(result))

### 3.2 PerformanceMeasureBase

In [ ]:
# PerformanceMeasureBase collects duration measurements and computes statistics.
# Durations are in nanoseconds.

pm = le_eval.PerformanceMeasureBase("windmill", "SobelGradient")
print(f"source_name: '{pm.source_name}', task_name: '{pm.task_name}'")
print(f"Initial durations: {pm.durations}")

# Simulate 10 timing measurements (in nanoseconds)
simulated_ns = [1_200_000, 1_350_000, 1_100_000, 1_500_000, 1_250_000,
                1_300_000, 1_400_000, 1_150_000, 1_280_000, 1_320_000]

for ns in simulated_ns:
    pm.append_duration(ns)

print(f"Durations: {len(pm.durations)} samples")

# Compute statistics
result = pm.compute_result()
print(f"\nResult:")
print(f"  Total:  {result.total:.0f} ns ({result.total / 1e6:.2f} ms)")
print(f"  Mean:   {result.mean:.0f} ns ({result.mean / 1e6:.2f} ms)")
print(f"  Stddev: {result.stddev:.0f} ns ({result.stddev / 1e6:.2f} ms)")

In [ ]:
# Clear resets all collected data
pm.clear()
print(f"After clear(): {len(pm.durations)} durations")

### 3.3 CVPerformanceMeasure

In [ ]:
# CVPerformanceMeasure extends PerformanceMeasureBase with image dimensions.
# This lets you compute throughput in megapixels/second.

m = le_eval.CVPerformanceMeasure("windmill", "LsdCC", 1920.0, 1080.0)
print(f"Dimensions: {m.width:.0f} × {m.height:.0f}")
print(f"Megapixels: {m.mega_pixels():.2f}")

# Inherit duration tracking from PerformanceMeasureBase
m.append_duration(5_000_000)  # 5 ms
m.append_duration(4_800_000)
m.append_duration(5_200_000)

result = m.compute_result()
print(f"\nMean time: {result.mean / 1e6:.2f} ms")
print(f"Throughput: {m.mega_pixels() / (result.mean / 1e9):.1f} MP/s")

### 3.4 Measure

In [ ]:
# Measure is a lightweight struct for labeling source/task pairs.
m = le_eval.Measure()
m.source_name = "bsds_001"
m.task_name = "LsdFGioi"
print(f"source='{m.source_name}', task='{m.task_name}'")

m.clear()
print(f"After clear: source='{m.source_name}'")

### 3.5 GenericInputData

In [ ]:
# GenericInputData is a named container for input data.
d = le_eval.GenericInputData("test_input")
print(f"Name: '{d.name}'")

d.name = "renamed"
print(f"Renamed: '{d.name}'")

---
## 4. Data Providers

Data providers supply images to the benchmarking framework. They iterate
over image files and wrap them in `CVData` or `CVPerformanceData` objects.

### 4.1 CVData and CVPerformanceData

In [ ]:
# CVData wraps an image with a name
d = le_eval.CVData()
d.name = "windmill"
d.src = np.zeros((100, 100, 3), dtype=np.uint8)  # Simulated image data

print(f"CVData: name='{d.name}', src shape={d.src.shape}")

# CVPerformanceData extends with metadata for benchmarking
ti = TestImages()
img = plt.imread(str(ti.windmill())).copy()  # .copy() ensures writable array
pd = le_eval.CVPerformanceData("windmill", img)
print(f"CVPerformanceData: name='{pd.name}'")

### 4.2 File-Based Data Providers

In [ ]:
# FileCVDataProvider loads images from a directory.
provider = le_eval.FileCVDataProvider("test_provider")
print(f"Provider name: '{provider.name}'")

# rewind() and clear() manage the iterator state
provider.rewind()
provider.clear()
print("Provider reset.")

In [ ]:
# FileCVPerformanceDataProvider is the performance-oriented variant
perf_provider = le_eval.FileCVPerformanceDataProvider("perf_provider")
print(f"Perf provider: '{perf_provider.name}'")
perf_provider.rewind()
perf_provider.clear()

In [ ]:
# FileDataProvider is a convenience wrapper that takes a directory path
with tempfile.TemporaryDirectory() as data_dir:
    fp = le_eval.FileDataProvider(data_dir, "temp_data")
    print(f"FileDataProvider: name='{fp.name}', dir='{data_dir}'")

---
## 5. Writing Custom Tasks

The evaluation framework lets you wrap any algorithm as a **task** that
can be benchmarked. There are two main approaches:

1. Subclass `Task` for generic tasks
2. Subclass `CVPerformanceTask` for image processing benchmarks

### Task Flags

| Flag | Value | Meaning |
|------|-------|---------|
| `TASK_SQR` | 1 | Convert input to grayscale |
| `TASK_RGB` | 2 | Keep input as RGB |
| `TASK_NO_3` | 4 | Disable 3-channel processing |
| `TASK_NO_5` | 8 | Disable 5-channel processing |

### 5.1 Basic Task Subclassing

In [ ]:
# A Task requires overriding run(loops)
class CountingTask(le_eval.Task):
    """Simple task that counts invocations."""
    
    def __init__(self, name: str) -> None:
        super().__init__(name)
        self.call_count = 0
    
    def run(self, loops: int) -> None:
        self.call_count += loops
    
    def reset(self) -> None:
        self.call_count = 0

task = CountingTask("counter")
print(f"Task name: '{task.name}', task_name(): '{task.task_name()}'")
print(f"Verbose: {task.verbose}")

# Verify it's an ITask
print(f"Is ITask: {isinstance(task, le_eval.ITask)}")

task.run(5)
task.run(3)
print(f"Call count: {task.call_count}")

task.reset()
print(f"After reset: {task.call_count}")

In [ ]:
# Task also supports save_visual_results() override
class SavingTask(le_eval.Task):
    def __init__(self, name: str) -> None:
        super().__init__(name)
        self.saved_path = None
    
    def run(self, loops: int) -> None:
        pass
    
    def save_visual_results(self, target_path: str) -> None:
        self.saved_path = target_path
        print(f"Would save results to: {target_path}")

st = SavingTask("saver")
st.save_visual_results("/tmp/results")
print(f"Saved path: {st.saved_path}")

### 5.2 CVPerformanceTask — Image Processing Benchmarks

`CVPerformanceTask` is the right base class for benchmarking image processing
algorithms. Override:

- `prepare_impl(src)` — one-time setup before timing (optional)
- `run_impl(name, src)` — the timed operation

In [ ]:
# Basic CVPerformanceTask: wraps a gradient filter
class GradientTask(le_eval.CVPerformanceTask):
    """Benchmark task for gradient computation."""
    
    def __init__(self, name: str, gradient_cls) -> None:
        # TASK_SQR flag = convert input to grayscale
        super().__init__(name, le_eval.TASK_SQR)
        self.gradient_cls = gradient_cls
        self.grad = None
    
    def prepare_impl(self, src: np.ndarray) -> None:
        # Create a fresh gradient filter for each image
        self.grad = self.gradient_cls()
    
    def run_impl(self, name: str, src: np.ndarray) -> None:
        self.grad.process(src)

# Test construction
gt = GradientTask("Sobel", le_imgproc.SobelGradient)
print(f"Task name: '{gt.name}'")
print(f"sqr(): {gt.sqr()}, rgb(): {gt.rgb()}")
print(f"border(): {gt.border()}")
print(f"verbose: {gt.verbose}")

In [ ]:
# Task flags control input preprocessing
print("Task flag constants:")
print(f"  TASK_SQR  = {le_eval.TASK_SQR}  (convert to grayscale)")
print(f"  TASK_RGB  = {le_eval.TASK_RGB}  (keep as RGB)")
print(f"  TASK_NO_3 = {le_eval.TASK_NO_3}  (disable 3-channel)")
print(f"  TASK_NO_5 = {le_eval.TASK_NO_5}  (disable 5-channel)")

# Combining flags
combined = le_eval.CVPerformanceTask("combined", le_eval.TASK_SQR | le_eval.TASK_NO_5)
print(f"\nCombined flags: sqr={combined.sqr()}, rgb={combined.rgb()}")

In [ ]:
# A more complete task: wrap an LSD detector
class LsdTask(le_eval.CVPerformanceTask):
    """Benchmark task for line segment detection."""
    
    def __init__(self, name: str, detector_cls, **kwargs) -> None:
        super().__init__(name, le_eval.TASK_SQR)
        self.detector_cls = detector_cls
        self.kwargs = kwargs
        self.det = None
        self.last_count = 0
    
    def prepare_impl(self, src: np.ndarray) -> None:
        self.det = self.detector_cls(**self.kwargs)
    
    def run_impl(self, name: str, src: np.ndarray) -> None:
        self.det.detect(src)
        self.last_count = len(self.det.line_segments())

# Create tasks for several detectors
tasks = [
    LsdTask("LsdCC", le_lsd.LsdCC),
    LsdTask("LsdFGioi", le_lsd.LsdFGioi),
    LsdTask("LsdEDLZ", le_lsd.LsdEDLZ),
    LsdTask("LsdHoughP", le_lsd.LsdHoughP),
]

for t in tasks:
    print(f"Task: '{t.name}', sqr={t.sqr()}")

### Exercise 3.2

Create a `CVPerformanceTask` subclass that benchmarks edge segment detection.
It should:
1. In `prepare_impl`, create an `EdgeSourceSobel` and an `EsdDrawing`
2. In `run_impl`, process the image with the edge source and detect segments
3. Store the number of detected segments in an instance variable

Test it manually by calling `prepare_impl` and `run_impl` directly.

In [ ]:
# TODO: Create an EsdTask subclass and test it.

**Solution**

In [ ]:
class EsdTask(le_eval.CVPerformanceTask):
    """Benchmark edge segment detection."""
    
    def __init__(self, name: str, esd_cls, **kwargs) -> None:
        super().__init__(name, le_eval.TASK_SQR)
        self.esd_cls = esd_cls
        self.kwargs = kwargs
        self.es = None
        self.esd = None
        self.segment_count = 0
    
    def prepare_impl(self, src: np.ndarray) -> None:
        self.es = le_edge.EdgeSourceSobel()
        self.esd = self.esd_cls(**self.kwargs)
    
    def run_impl(self, name: str, src: np.ndarray) -> None:
        self.es.process(src)
        self.esd.detect(self.es)
        self.segment_count = len(self.esd.segments())

# Manual test
task = EsdTask("EsdDrawing", le_edge.EsdDrawing, min_pixels=5)

# Load a test image
ti = TestImages()
img = plt.imread(str(ti.windmill())).copy()
if img.ndim == 3:
    gray = np.dot(img[..., :3], [0.2989, 0.5870, 0.1140]).astype(np.uint8)
else:
    gray = img.copy()

task.prepare_impl(gray)
task.run_impl("windmill", gray)
print(f"EsdTask detected {task.segment_count} segments")

---
## 6. CVPerformanceTest — The Orchestrator

`CVPerformanceTest` ties everything together: it runs multiple tasks across
multiple data providers and collects timing results into a `StringTable`.

**Constructor:** `CVPerformanceTest(providers, name)`

**Methods:**
- `.add_task(task)` — register a task
- `.result_table()` → `StringTable` with timing results
- `.clear()` — reset all results

**Display toggles:**
- `.show_total` — include total time
- `.show_mean` — include mean time (default: True)
- `.show_std_dev` — include standard deviation
- `.show_mega_pixel` — include megapixel throughput

In [ ]:
# Basic orchestrator setup
providers = []  # Empty for now
bench = le_eval.CVPerformanceTest(providers, "demo_benchmark")

print(f"show_total: {bench.show_total}")
print(f"show_mean: {bench.show_mean}")
print(f"show_std_dev: {bench.show_std_dev}")
print(f"show_mega_pixel: {bench.show_mega_pixel}")

# Toggle display options
bench.show_total = True
bench.show_std_dev = True
bench.show_mega_pixel = True

# Add tasks
bench.add_task(le_eval.CVPerformanceTask("dummy_task", 0))

# Get result table (empty since no data was processed)
table = bench.result_table()
print(f"\nResult table: {table.rows}×{table.cols}")
print(repr(table))

bench.clear()
print("\nBenchmark cleared.")

---
## 7. Accumulating Measures

The `accumulate_measures()` function combines multiple `CVPerformanceMeasure`
objects — useful for aggregating results across multiple images.

In [ ]:
# Create measures for different images
measures = []
for i, (w, h) in enumerate([(640, 480), (1920, 1080), (800, 600)]):
    m = le_eval.CVPerformanceMeasure(f"image_{i}", "LsdCC", float(w), float(h))
    # Add some simulated durations
    for _ in range(5):
        duration = int((w * h * 0.01 + np.random.normal(0, 100)) * 1000)  # ns
        m.append_duration(max(1, duration))
    r = m.compute_result()
    measures.append(m)
    print(f"Image {i}: {w}×{h} = {m.mega_pixels():.2f} MP, "
          f"mean={r.mean/1e6:.2f} ms")

# Accumulate
accumulated = le_eval.accumulate_measures(measures)
print(f"\nAccumulated: {len(accumulated.durations)} durations, "
      f"{accumulated.mega_pixels():.2f} MP")

acc_result = accumulated.compute_result()
print(f"Accumulated mean: {acc_result.mean/1e6:.2f} ms")

In [ ]:
# Accumulating empty list
empty_acc = le_eval.accumulate_measures([])
print(f"Empty accumulation: {empty_acc.mega_pixels():.2f} MP")

---
## 8. Full Benchmark Example

Let's put everything together and run a complete benchmark comparing
multiple LSD detectors across several images.

In [ ]:
# Load test images
import cv2

ti = TestImages()

# Windmill
wm = cv2.imread(str(ti.windmill()), cv2.IMREAD_GRAYSCALE)

# A few BSDS500 images
bsds_iter = ti.bsds500()
bsds_images = []
for _ in range(3):
    try:
        path = next(bsds_iter)
        img = cv2.imread(str(path), cv2.IMREAD_GRAYSCALE)
        bsds_images.append((path.stem, img))
    except StopIteration:
        break

# Collect all test images
test_images = [("windmill", wm)] + bsds_images
print(f"Loaded {len(test_images)} test images:")
for name, img in test_images:
    print(f"  {name}: {img.shape}")

In [ ]:
# Define detectors to benchmark
detector_config = [
    ("LsdCC", le_lsd.LsdCC, {}),
    ("LsdCP", le_lsd.LsdCP, {}),
    ("LsdBurns", le_lsd.LsdBurns, {}),
    ("LsdFBW", le_lsd.LsdFBW, {}),
    ("LsdFGioi", le_lsd.LsdFGioi, {}),
    ("LsdEDLZ", le_lsd.LsdEDLZ, {}),
    ("LsdEL", le_lsd.LsdEL, {}),
    ("LsdEP", le_lsd.LsdEP, {}),
    ("LsdHoughP", le_lsd.LsdHoughP, {}),
]

N_LOOPS = 3

# Run benchmark
results = {}  # {detector_name: {image_name: (n_segments, mean_time_ms)}}

for det_name, det_cls, kwargs in detector_config:
    results[det_name] = {}
    for img_name, img in test_images:
        det = det_cls(**kwargs)
        
        # Warm up
        det.detect(img)
        n_segs = len(det.line_segments())
        
        # Time it
        times = []
        for _ in range(N_LOOPS):
            t0 = time.perf_counter_ns()
            det.detect(img)
            t1 = time.perf_counter_ns()
            times.append((t1 - t0) / 1e6)  # ms
        
        mean_ms = np.mean(times)
        results[det_name][img_name] = (n_segs, mean_ms)

print("Benchmark complete.")

In [ ]:
# Build a StringTable from results
img_names = [name for name, _ in test_images]
det_names = [name for name, _, _ in detector_config]

# Table: segments per detector per image
n_rows = len(det_names) + 1
n_cols = len(img_names) + 1

seg_table = le_eval.StringTable(n_rows, n_cols)
seg_table[0, 0] = "Detector"
for c, iname in enumerate(img_names):
    seg_table[0, c + 1] = iname

for r, dname in enumerate(det_names):
    seg_table[r + 1, 0] = dname
    for c, iname in enumerate(img_names):
        n_segs, _ = results[dname][iname]
        seg_table[r + 1, c + 1] = str(n_segs)

print("Segments detected:")
print(repr(seg_table))

# Time table
time_table = le_eval.StringTable(n_rows, n_cols)
time_table[0, 0] = "Detector"
for c, iname in enumerate(img_names):
    time_table[0, c + 1] = iname

for r, dname in enumerate(det_names):
    time_table[r + 1, 0] = dname
    for c, iname in enumerate(img_names):
        _, ms = results[dname][iname]
        time_table[r + 1, c + 1] = f"{ms:.2f}"

print("\nDetection time (ms):")
print(repr(time_table))

In [ ]:
# Save results to CSV
csv_seg = os.path.join(tempfile.gettempdir(), "le_segments.csv")
csv_time = os.path.join(tempfile.gettempdir(), "le_times.csv")

seg_table.save_csv(csv_seg)
time_table.save_csv(csv_time)

print(f"Saved: {csv_seg}")
print(f"Saved: {csv_time}")

# Show CSV content
with open(csv_time) as f:
    print(f"\nTime CSV:\n{f.read()}")

In [ ]:
# Visualization: bar chart per image
fig, axes = plt.subplots(1, len(img_names), figsize=(6 * len(img_names), 5))
if len(img_names) == 1:
    axes = [axes]

for ax, iname in zip(axes, img_names):
    segs = [results[d][iname][0] for d in det_names]
    ax.barh(det_names, segs, color="steelblue")
    ax.set_xlabel("Segments")
    ax.set_title(f"{iname}")
    ax.invert_yaxis()

plt.suptitle("Segments Detected per Image", fontsize=14)
plt.tight_layout()
plt.show()

In [ ]:
# Visualization: timing comparison (average across all images)
avg_times = []
avg_segs = []
for dname in det_names:
    times = [results[dname][iname][1] for iname in img_names]
    segs = [results[dname][iname][0] for iname in img_names]
    avg_times.append(np.mean(times))
    avg_segs.append(np.mean(segs))

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

ax1.barh(det_names, avg_times, color="coral")
ax1.set_xlabel("Mean Time (ms)")
ax1.set_title("Average Detection Time")
ax1.invert_yaxis()

ax2.barh(det_names, avg_segs, color="seagreen")
ax2.set_xlabel("Mean Segments")
ax2.set_title("Average Segments Detected")
ax2.invert_yaxis()

plt.tight_layout()
plt.show()

In [ ]:
# Scatter: time vs segments (speed vs quantity trade-off)
plt.figure(figsize=(8, 6))
for i, dname in enumerate(det_names):
    plt.scatter(avg_times[i], avg_segs[i], s=100, zorder=5)
    plt.annotate(dname, (avg_times[i], avg_segs[i]),
                 textcoords="offset points", xytext=(5, 5), fontsize=9)

plt.xlabel("Mean Detection Time (ms)")
plt.ylabel("Mean Segments Detected")
plt.title("Speed vs. Quantity Trade-Off")
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

### Exercise 3.3

Extend the benchmark:

1. Create a `CVPerformanceMeasure` for each (detector, image) pair
2. Accumulate the measures per detector using `accumulate_measures()`
3. Compute throughput in megapixels/second for each detector
4. Create a bar chart showing MP/s for each detector

In [ ]:
# TODO: Build CVPerformanceMeasures, accumulate, compute throughput, plot.

**Solution**

In [ ]:
throughputs = []

for det_name, det_cls, kwargs in detector_config:
    measures = []
    for img_name, img in test_images:
        h, w = img.shape[:2]
        m = le_eval.CVPerformanceMeasure(img_name, det_name, float(w), float(h))
        
        det = det_cls(**kwargs)
        for _ in range(N_LOOPS):
            t0 = time.perf_counter_ns()
            det.detect(img)
            t1 = time.perf_counter_ns()
            m.append_duration(t1 - t0)
        
        measures.append(m)
    
    acc = le_eval.accumulate_measures(measures)
    r = acc.compute_result()
    mp_per_sec = acc.mega_pixels() / (r.mean / 1e9) if r.mean > 0 else 0
    throughputs.append(mp_per_sec)
    print(f"{det_name:12s}: {mp_per_sec:.1f} MP/s (mean: {r.mean/1e6:.2f} ms)")

plt.figure(figsize=(10, 5))
plt.barh(det_names, throughputs, color="teal")
plt.xlabel("Throughput (Megapixels / second)")
plt.title("Detector Throughput")
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()

Throughput normalizes for image size, making it possible to fairly compare
performance across images of different resolutions.

### Exercise 3.4

Benchmark the **noise robustness** of `LsdEL`:

1. Load the bike image and its noise variants from `TestImages.noise()`
2. Run `LsdEL` on each, collecting timing with `CVPerformanceMeasure`
3. Plot noise level vs. (a) detection time and (b) segments detected

In [ ]:
# TODO: Load noise images, benchmark LsdEL on each, plot results.

**Solution**

In [ ]:
ti = TestImages()

noise_names = ["bike", "bike_noise10", "bike_noise20", "bike_noise30", "bike_noise50"]
noise_images = {}
for name in noise_names:
    img = cv2.imread(str(ti.noise(f"{name}.png")), cv2.IMREAD_GRAYSCALE)
    noise_images[name] = img

# Benchmark
noise_times = []
noise_segs = []

for name, img in noise_images.items():
    h, w = img.shape[:2]
    m = le_eval.CVPerformanceMeasure(name, "LsdEL", float(w), float(h))
    
    det = le_lsd.LsdEL()
    for _ in range(5):
        t0 = time.perf_counter_ns()
        det.detect(img)
        t1 = time.perf_counter_ns()
        m.append_duration(t1 - t0)
    
    r = m.compute_result()
    n = len(det.line_segments())
    noise_times.append(r.mean / 1e6)
    noise_segs.append(n)
    print(f"{name:16s}: {n:4d} segs, {r.mean/1e6:.2f} ms")

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))

ax1.plot(noise_names, noise_times, "o-", color="coral")
ax1.set_ylabel("Time (ms)")
ax1.set_title("Detection Time vs. Noise")
ax1.tick_params(axis="x", rotation=45)

ax2.plot(noise_names, noise_segs, "o-", color="steelblue")
ax2.set_ylabel("Segments")
ax2.set_title("Segments vs. Noise")
ax2.tick_params(axis="x", rotation=45)

plt.tight_layout()
plt.show()

Noise typically increases both detection time (more candidate edges to process)
and segment count (spurious short detections). At very high noise, some detectors
may actually slow down due to the larger number of seed points.

---
## Summary

In this tutorial you learned:

- **StringTable**: Structured result storage with CSV export for reproducible
  reporting
- **Performance Primitives**: `PerformanceResult`, `PerformanceMeasureBase`, and
  `CVPerformanceMeasure` for collecting and computing timing statistics
- **Data Providers**: File-based image loading for systematic benchmarking
- **Custom Tasks**: Wrapping any algorithm as a `CVPerformanceTask` for
  integration with the benchmarking framework
- **CVPerformanceTest**: The orchestrator for running multi-task, multi-image
  benchmarks
- **Result Analysis**: Accumulating measures across images, computing throughput,
  and creating publication-quality comparison charts

### Full Tutorial Series

- **CV Primer** — Image processing fundamentals for beginners
- **Tutorial 1** — Library fundamentals (`le_imgproc`, `le_geometry`, `TestImages`)
- **Tutorial 2** — Edge & line detection pipelines (`le_edge`, `le_lsd`)
- **Tutorial 3** — Performance evaluation framework (`le_eval`) ← you are here